### Import the requirements librarys
- Slenium for Web Sraping and Open Browser
- time for wait for load the pages
- unidecode for Convert Persian numbers to English
- Bs4 for pulling data out of HTML
- Database is file in this directory

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.common.keys import Keys
from unidecode import unidecode
from bs4 import BeautifulSoup
from os import system
from DataBase import Phones, session

system("clear")

2022-08-25 12:54:39,461 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-08-25 12:54:39,463 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Phones")
2022-08-25 12:54:39,465 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-25 12:54:39,467 INFO sqlalchemy.engine.Engine COMMIT


1

- The function checks whether the element is on the page or not

In [4]:
def check_element_exist(mode, inp):
    try:
        return driver.find_element(mode, inp)
    except:
        return False

- This is function for load the digikala pages 
- This function looks for pagination because pagination is loaded when the entire content is loaded

In [5]:
def load_all_page(url):
    print("[x] Waiting For Load Page ", url) ## LOG
    driver.get(url)
    body = driver.find_element("tag name", "html")
    pagination = check_element_exist("xpath", "/html/body/div[1]/div[1]/div[3]/div[3]/div[1]/div/section[1]/div[2]/div[2]/div[2]")
    while not pagination:
        print("[x] Try to Loading Page", url) ## LOG
        body.send_keys(Keys.END)
        sleep(2)
        pagination = check_element_exist("xpath", "/html/body/div[1]/div[1]/div[3]/div[3]/div[1]/div/section[1]/div[2]/div[2]/div[2]")
    sleep(3)
    print(f"[+] Page {url} Loaded") ## LOG

- This function is used to shorten the Url of the product image

In [6]:
def shorten_link(link):
    valid = link.split("/")[:3]
    return "/".join(valid)

- This function extracts the elements of all the products on the page

In [7]:
def extract_products():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    source = soup.find("div", {"class": "d-flex flex-wrap"})
    return source.find_all("div", {"class": "border-b border-l"})

- This function is used to extract detailed product information and checks if the product price element is "ناموجود" (unavailable)and exits the program if it is not available

In [8]:
def extract_product_information(products):
    objects = []
    for product in products:
        link = shorten_link(product.find("a")["href"])
        image = product.find("img")["data-src"]
        name = product.find("h2").text
        try:
            price = product.find(class_="d-flex ai-center jc-end gap-1 color-700 color-400 text-h5 grow-1").span.text
            objects.append(Phones(link=link, photo=image, name=name, price=unidecode(price)))
            system("clear")
        except:
            return False
    session.add_all(objects)
    session.commit()
    system("clear")

    print("[+] Add ", len(objects), "Objects in DataBase") ## LOG

- This function connects all program functions together
First, the user adds the first page to the input link and starts loading the page and products 

In [16]:
def main(url):
    ## Example Url "https://www.digikala.com/search/category-notebook-netbook-ultrabook/"
    cant = open("cant.txt", "a")
    for i in range(1, 100):
        url_p = url+"?page="+str(i)
        try:
            load_all_page(url=url_p)
            products = extract_products()
            output = extract_product_information(products)
            if output == False:
                print("[+] Products Ended") ## LOG
                break
        except Exception as err:
            cant.write(url+"\n")
            print("[-] Error On Page ", url_p)

- And here we get input from the user and open the browser and load the url

In [17]:
url = input("Enter The List Of Product Like This Link:\n[https://www.digikala.com/search/category-notebook-netbook-ultrabook/]\nLink: ")
system("clear")
print("[x] Waiting For Load Driver . . .") ## LOG
driver = webdriver.Firefox(executable_path="C://geckodriver.exe")
system("clear")
main(url)

[x] Waiting For Load Driver . . .


C:\Users\user\AppData\Local\Temp\ipykernel_11136\3891079766.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path="C://geckodriver.exe")


[x] Waiting For Load Page  https://www.digikala.com/search/category-mobile-phone/product-list/?page=1
[+] Page https://www.digikala.com/search/category-mobile-phone/product-list/?page=1 Loaded
2022-08-25 13:02:09,944 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-08-25 13:02:09,948 INFO sqlalchemy.engine.Engine INSERT INTO "Phones" (link, photo, name, price) VALUES (?, ?, ?, ?)
2022-08-25 13:02:09,950 INFO sqlalchemy.engine.Engine [cached since 134.7s ago] ('/product/dkp-6460974', '/statics/img/svg/productCard/topBadge/SpecialSell.svg', 'گوشی موبایل اپل مدل iPhone 13 Pro Max A2644 دو سیم\u200c کارت ظرفیت 256 گیگابایت و رم 6 گیگابایت', '53,389,000')
2022-08-25 13:02:09,958 INFO sqlalchemy.engine.Engine INSERT INTO "Phones" (link, photo, name, price) VALUES (?, ?, ?, ?)
2022-08-25 13:02:09,961 INFO sqlalchemy.engine.Engine [cached since 134.7s ago] ('/product/dkp-7845430', 'https://dkstatics-public.digikala.com/digikala-products/e477016feaff5a6e6cf930517134eb3888c887ef_1656404674.jp